## Python Importation

In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import bk.compute
import bk.plot
import bk.load

import scipy.signal
import neuroseries as nts

## Magic Command

In [2]:
%matplotlib qt

## Session Init

In [3]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

In [14]:
bk.load.current_session()

FileNotFoundError: [Errno 2] No such file or directory: 'Z:/All-Rats/Billel/session_indexing.csv'

## Loading data

In [4]:
neurons, metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()

Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


## Computing spectrograms

In [6]:
start = 15516.0
end = 16405.0

# start = 16120
# end = 16200
interval = nts.IntervalSet(start,end,time_units='s')

In [7]:
lfp = bk.load.lfp(start,end,166,13)
f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=10000,noverlap=5000)
t = t +lfp.as_units('s').index[0]
spec = Sxx[f<20]
spec_corr = np.corrcoef(spec,rowvar=False)

Load LFP from Rat08-20130713.lfp


## Computing Neurons correlations

In [8]:
# start = 15516.0
# end = 16405.0

start = 2300
end = 2500

# start = 16120
# end = 16200
interval = nts.IntervalSet(start,end,time_units='s')

# stru = ['Hpc','BLA','Pir']
# channel = [13,101,80]

stru =  ['BLA']
channel = [103]
for n,(s,c) in enumerate(zip(stru,channel),0):
    lfp = bk.load.lfp(start,end,166,c)
    f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=1000,noverlap=250)
    t = t +lfp.as_units('s').index[0]
    spec = Sxx[f<100]
    spec_corr = np.corrcoef(spec,rowvar=False)
    
    stru_neurons,t_n = bk.compute.binSpikes(neurons[metadata.Region == s],2,start,end)
    neurons_corr = np.corrcoef(stru_neurons,rowvar=False)
    
    plt.figure()
    sub = plt.subplot(4,1,1)
    plt.title('LFP')
    plt.plot(lfp.as_units('s'))
    bk.plot.intervals(states['sws'],col='grey')
    bk.plot.intervals(states['Rem'])
   
    plt.subplot(4,1,2,sharex=sub)
    plt.title('Spectrogram')
    plt.pcolormesh(t, f, Sxx)
    plt.ylim(0,100)
    plt.clim(0,5000)
    
    
    plt.subplot(4,1,3,sharex=sub)
    plt.title('SpectrogramCorr')
    plt.pcolormesh(t,t,spec_corr)
    plt.clim(0,1)
    
    plt.subplot(4,1,4,sharex = sub)
    plt.title('SpikeTrainCorr') 
    plt.pcolormesh(t_n,t_n,neurons_corr)
    plt.xlim(start,end)
    
    plt.tight_layout()
    plt.suptitle(s)

Load LFP from Rat08-20130713.lfp


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.


In [32]:
stru_neurons.shape

(94, 999)

In [10]:
fig, axes = plt.subplots(nrows=3, ncols=2)

In [16]:
for ax in axes.flatten():
    ax.plot(lfp)

In [31]:
states['sws'].end

0      1697000000
1      1926000000
2      2429000000
3      3275000000
4      3841000000
5      3916000000
6      3992000000
7      4529000000
8      4648000000
9      5426000000
10     6069000000
11     6629000000
12     7311000000
13     7759000000
14     7909000000
15     8797000000
16     9333000000
17    16145000000
18    16765000000
19    17259000000
20    17429000000
21    17669000000
22    17764000000
23    17810000000
24    18076000000
25    18528000000
26    18979000000
27    19167000000
28    19448000000
29    19771000000
30    20363000000
31    21286000000
32    21629000000
33    21850000000
Name: end, dtype: int64

In [9]:
end  = nts.Ts(np.array(states['sws'].end+1_000_001))
a = states['Rem'].in_interval(end)

In [10]:
states['sws'].iloc[1]

start    1777000000
end      1926000000
Name: 1, dtype: int64

In [11]:
states['Rem'].iloc[0]

start    1927000000
end      2013000000
Name: 0, dtype: int64

In [12]:
interval.

SyntaxError: invalid syntax (<ipython-input-12-5ceda6e46fc5>, line 1)

In [14]:
import itertools

In [15]:
for j in itertools.permutations(states.keys(),2):

SyntaxError: unexpected EOF while parsing (<ipython-input-15-fdc0e234b070>, line 1)

In [30]:
def transitions_times(states,epsilon = 1):
    '''
        states : dict of nts.Interval_Set
        
        This function compute transition in between Intervals in a dict.
        It returns a new dict with intervals and when the transition occurs
        
        epsilon : tolerance time delay between state
        
    '''
    
    import itertools
    
    empty_state = []
    for state in states:
        if len(states[state]) == 0:
            empty_state.append(state)
            continue
        states[state] = states[state].drop_short_intervals(1)
    
    
    for i in empty_state: del states[i]
        
    transitions_intervals = {}
    transitions_timing = {}
    
    for items in itertools.permutations(states.keys(),2):
#         states[items[0]] = states[items[0]].drop_short_intervals(1)
#         states[items[1]] = states[items[1]].drop_short_intervals(1)
        
        print('Looking at transition from',items[0],' to ',items[1])
        end = nts.Ts(np.array(states[items[0]].end + (epsilon * 1_000_000)+1))
        in_next_epoch = states[items[1]].in_interval(end)
        
        transitions_intervals.update({items:[]})
        transitions_timing.update({items:[]})

        for n,t in enumerate(in_next_epoch):
            if np.isnan(t): continue            
            start = states[items[0]].iloc[n].start
            trans = int(np.mean([states[items[0]].iloc[n].end,states[items[0]].iloc[n].start]))
            end  = states[items[1]].iloc[int(t)].end
            transitions_intervals[items].append([start,end])
            transitions_timing[items].append(trans)
        
        if  not transitions_timing[items] == []:      
            transitions_intervals[items] = np.array(transitions_intervals[items])
            transitions_intervals[items] = nts.IntervalSet(transitions_intervals[items][:,0],transitions_intervals[items][:,1])
            
            transitions_timing[items] = nts.Ts(t = np.array(transitions_timing[items]))
    return transitions_intervals,transitions_timing

In [5]:
a,b = bk.compute.transitions_times(states)
full = {**a,**states}
c,d = bk.compute.transitions_times(full)

Looking at transition from Rem  to  sws
Looking at transition from Rem  to  wake
Looking at transition from Rem  to  drowsy
Looking at transition from sws  to  Rem
Looking at transition from sws  to  wake
Looking at transition from sws  to  drowsy
Looking at transition from wake  to  Rem
Looking at transition from wake  to  sws
Looking at transition from wake  to  drowsy
Looking at transition from drowsy  to  Rem
Looking at transition from drowsy  to  sws
Looking at transition from drowsy  to  wake
Looking at transition from ('Rem', 'sws')  to  ('Rem', 'wake')
Looking at transition from ('Rem', 'sws')  to  ('Rem', 'drowsy')
Looking at transition from ('Rem', 'sws')  to  ('sws', 'Rem')
Looking at transition from ('Rem', 'sws')  to  ('sws', 'wake')
Looking at transition from ('Rem', 'sws')  to  ('sws', 'drowsy')
Looking at transition from ('Rem', 'sws')  to  ('wake', 'sws')
Looking at transition from ('Rem', 'sws')  to  ('wake', 'drowsy')
Looking at transition from ('Rem', 'sws')  to  ('

/home/billel/Documents/Python/Amy-Hpc-sleep-dynamics-python/neuroseries/interval_set.py:65: UserWarning: some start precede the previous end
  warn(msg, UserWarning)


Looking at transition from ('Rem', 'drowsy')  to  Rem
Looking at transition from ('Rem', 'drowsy')  to  sws
Looking at transition from ('Rem', 'drowsy')  to  wake
Looking at transition from ('Rem', 'drowsy')  to  drowsy
Looking at transition from ('sws', 'Rem')  to  ('Rem', 'sws')
Looking at transition from ('sws', 'Rem')  to  ('Rem', 'wake')
Looking at transition from ('sws', 'Rem')  to  ('Rem', 'drowsy')
Looking at transition from ('sws', 'Rem')  to  ('sws', 'wake')
Looking at transition from ('sws', 'Rem')  to  ('sws', 'drowsy')
Looking at transition from ('sws', 'Rem')  to  ('wake', 'sws')
Looking at transition from ('sws', 'Rem')  to  ('wake', 'drowsy')
Looking at transition from ('sws', 'Rem')  to  ('drowsy', 'Rem')
Looking at transition from ('sws', 'Rem')  to  ('drowsy', 'sws')
Looking at transition from ('sws', 'Rem')  to  ('drowsy', 'wake')
Looking at transition from ('sws', 'Rem')  to  Rem
Looking at transition from ('sws', 'Rem')  to  sws
Looking at transition from ('sws', 

In [9]:
s = ('sws','Rem')

ax = plt.subplot(2,1,1)
bk.plot.intervals(c[s],col = 'red')
plt.subplot(2,1,2,sharex=ax)
bk.plot.intervals(states['Rem'],col = 'orange')
bk.plot.intervals(states['sws'],col ='grey')
bk.plot.intervals(states['wake'],col = 'green')
for i in d[s].as_units('s').index:
    print(i)
    plt.axvline(i,c = 'r')

1926.5
2429.5
3275.5
3841.5
3916.5
3992.5
4529.5
5426.5
6069.5
7311.5
7759.5
7909.5
8797.5
9333.5
16145.5
16765.5
17259.5
17429.5
17810.5
18076.5
18528.5
18979.5
19167.5
19448.5
19771.5
21286.5
21629.5
21850.5


In [45]:
start = [1,10,25,30,50]
end = [2,12,35,40,55]

nts.IntervalSet(start,end,expect_fix=True)

,start,end
0,1,2
1,10,12
2,30,35
3,50,55
